## Notebook 5: Regresión Logística - Un tutorial sobre Stochastic Gradient Descent

## Referencias

Para completar el trabajo de este notebook, se recomienda ver el material siguiente:
- Videos Youtube: Curso de Andrew Ng sobre la Regresion Logística
    - https://www.youtube.com/watch?v=-la3q9d7AKQ (Lecture 6.1 — Logistic Regression | Classification : 8 minutes)
    - https://www.youtube.com/watch?v=t1IT5hZfS48 (Lecture 6.2 — Logistic Regression | Hypothesis Representation : 7 minutes)
    - https://www.youtube.com/watch?v=F_VG4LNjZZw (Lecture 6.3 — Logistic Regression | Decision Boundary : 14 minutes)
    - https://www.youtube.com/watch?v=HIQlmHxI6-0 (Lecture 6.4 — Logistic Regression | Cost Function : 11 minutes)
    - https://www.youtube.com/watch?v=TTdcc21Ko9A (Lecture 6.5 — Logistic Regression | Simplified Cost Function And Gradient Descent : 10 minutes)

## Introducción

La regresión logística es el algoritmo de clasificación lineal para problemas de dos clases.

En este tutorial, descubrirá cómo implementar la regresión logística con el descenso de gradiente estocástico (stochastic gradient descent) desde cero con Python.

Después de completar este tutorial, usted sabrá:

- Cómo hacer predicciones con un modelo de regresión logística.
- Cómo estimar los coeficientes utilizando el descenso de gradiente estocástico.
- Cómo aplicar la regresión logística a un problema real de predicción.


## Descripción

Esta sección dará una breve descripción de la técnica de regresión logística, el descenso por gradiente estocástico y el dataset "diabetes" que utilizaremos en este tutorial.

## Logistic Regression

La regresión logística se denomina así por la función utilizada en el núcleo del método, la función logística.

La regresión logística utiliza una ecuación como representación, muy similar a la regresión lineal. Los valores de entrada (X) se combinan linealmente utilizando pesos o valores de coeficiente para predecir un valor de salida (y).

Una diferencia clave con respecto a la regresión lineal es que el valor de salida que se está modelando es un valor binario (0 ó 1) en lugar de un valor numérico.

<code>yhat = e^(b0 + b1 * x1) / (1 + e^(b0 + b1 * x1))</code>

Esto puede simplificarse como:

<code>yhat = 1.0 / (1.0 + e^(-(b0 + b1 * x1)))</code>

Donde <b>e</b> es la base de los logaritmos naturales (el número de Euler), <b>yhat</b> es el resultado previsto, <b>b0</b> es el sesgo o término de intercepción y <b>b1</b> es el coeficiente para el valor de entrada simple (<b>x1</b>).

La predicción <b>yhat</b> es un valor real entre 0 y 1, que necesita ser redondeado a un valor entero y mapeado a un valor de clase predicho.

Cada columna de los datos de entrada tiene un coeficiente <b>b</b> asociado (un valor real constante) que debe aprenderse de los datos de entrenamiento. La representación real del modelo que almacenarías en memoria o en un archivo son los coeficientes de la ecuación.

Los coeficientes del algoritmo de regresión logística deben estimarse a partir de los datos de entrenamiento.

## Descenso estocástico por gradiente (Stochastic Gradient Descent)

El descenso por gradiente es el proceso de minimizar una función siguiendo los gradientes de la función de coste.

Esto implica conocer la forma del coste así como la derivada para que desde un punto dado conozcas la pendiente y puedas moverte en esa dirección, por ejemplo, cuesta abajo hacia el valor mínimo.

En el aprendizaje automático, podemos utilizar una técnica que evalúa y actualiza los coeficientes en cada iteración llamada descenso estocástico de gradiente para minimizar el error de un modelo en nuestros datos de entrenamiento.

La forma en que funciona este algoritmo de optimización es que cada instancia de entrenamiento se muestra al modelo una por una. El modelo hace una predicción para una instancia de entrenamiento, se calcula el error y se actualiza el modelo para reducir el error para la siguiente predicción.

Este procedimiento se puede utilizar para encontrar el conjunto de coeficientes en un modelo que da como resultado el menor error para el modelo en los datos de entrenamiento. En cada iteración, los coeficientes (b) se actualizan utilizando la ecuación:

<code>b = b + learning_rate * (y - yhat) * yhat * (1 - yhat) * x</code>

Donde <b>b</b> es el coeficiente o peso que se está optimizando, <b>learning_rate</b> es una velocidad de aprendizaje que se debe configurar (por ejemplo, 0.01), <b>(y - yhat)</b> es el error de predicción para el modelo en los datos de entrenamiento atribuidos al peso, <b>yhat</b> es la predicción hecha por los coeficientes y <b>x</b> es el valor de entrada.

## Dataset y Problema de clasificación

El dataset "diabetes.csv" consiste en predecir el inicio de la diabetes en un plazo de 5 años en los indios Pima a los que se les dan detalles médicos básicos.

Es un problema de clasificación binaria, donde la predicción es 0 (sin diabetes) o 1 (diabetes).

Contiene 768 filas y 9 columnas. Todos los valores en el archivo son numéricos, específicamente valores en coma flotante. Abajo hay una pequeña muestra de las primeras filas del problema.

<code>6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
...</code>

## Tutorial

Este tutorial está dividido en 3 partes.

    - Hacer predicciones.
    - Estimar los Coeficientes de la regresión logística
    - Predecir el diabete.

Esto le proporcionará la base que necesita para implementar y aplicar la regresión logística con descenso de gradiente estocástico sobre sus propios problemas de modelado predictivo.

### 1. Hacer predicciones

El primer paso es desarrollar una función que pueda hacer predicciones.

Esto será necesario tanto en la evaluación de los valores de los coeficientes candidatos en el descenso del gradiente estocástico como después de que se finalice el modelo y queramos empezar a hacer predicciones sobre los datos de prueba o nuevos datos.

Abajo hay una función llamada predict() que predice un valor de salida para una fila con un conjunto de coeficientes.

El primer coeficiente en es siempre la intercepción, también llamado el sesgo o b0 ya que es independiente y no es responsable de un valor de entrada específico.

In [ ]:
from math import exp

# Make a prediction with coefficients
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-yhat))

Podemos crear un pequeño conjunto de datos para probar nuestra función predict().

In [ ]:
# test predictions
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1],
]

In [ ]:
import pandas as pd

In [ ]:
url = 'example.csv'
example = pd.read_csv(url)
example.columns = ['x1','x2','y']
example

In [ ]:
url2 = 'example2.csv'
example2 = pd.read_csv(url2)
example2.columns = ['x1','x2','y']
example2

In [ ]:
X1 = example['x1']
X2 = example['x2']
X11 = example2['x1']
X22 = example2['x2']

<b>PREGUNTA:</b>

con Pandas y Matplotlib, construir un gráfico en dos dimensiones (x1 y x2), posicionando cada punto del dataset y diferenciando los puntos de cada clase.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(X1, X2,  marker='^', c='red')
plt.scatter(X11, X22,  marker='o', c='blue')

#plt.scatter(p1, p2)
plt.show()

Podemos usar coeficientes (b0, b1, b2) previamente preparados para hacer predicciones para este conjunto de datos:

In [ ]:
coef = [-0.406605464, 0.852573316, -1.104746259]

Poniendo todo esto junto podemos probar nuestra función predict() a continuación.

In [ ]:
for row in dataset:
    yhat = predict(row, coef)
    print("Expected=%.3f, Predicted=%.3f [%d]" % (row[-1], yhat, round(yhat)))

<b>PREGUNTA:</b> ¿Cuál es la precisión y el recall de nuestro modelo?

<h3><b>Respuesta</b></h3>
Si asumimos que los valores predecidos se aproximan de modo que todos los números que sean menores que 0.5 son iguales a 0 y los mayores o iguales a 0.5 son iguales a 1, la precisión y el recall son del 100%.

Por supuesto, en la realidad no podemos adivinar los buenos coeficientes b. Necesitamos un metodo para aproximar los mejores coeficientes. Presentamos a continuación el método Stochastic Gradient Descent que apunta a este objetivo. 

## 2. Estimación de los coeficientes

Podemos estimar los valores del coeficiente para nuestros datos de entrenamiento utilizando el descenso de gradiente estocástico.

El descenso estocástico por gradiente requiere dos parámetros:

    - Tasa de Aprendizaje (Learning rate): Se utiliza para limitar el importe que cada coeficiente se corrige cada vez que se actualiza.
    La cantidad que se le aumenta a cada coeficiente en cada iteracion.
    - Épocas (Epochs): El número de veces que se repasan los datos de entrenamiento mientras se actualizan los coeficientes.
    La cantidad de veces que se itera sobre el conjunto de dataset ompleto

Estos, junto con los datos de la formación, serán los argumentos de la función.

Hay 3 bucles que necesitamos realizar en la función:

    - Bucle sobre cada época.
    - Bucle sobre cada fila de los datos de entrenamiento de una época.
    - Bucle sobre cada coeficiente y actualícelo para una fila en una época.

Como puedes ver, actualizamos cada coeficiente para cada fila de los datos de entrenamiento, cada época.

Los coeficientes se actualizan en función del error que cometió el modelo. El error se calcula como la diferencia entre el valor de salida esperado y la predicción realizada con los coeficientes candidatos.

Hay un coeficiente para ponderar cada atributo de entrada, y éstos se actualizan de forma coherente, por ejemplo:

<code>b1(t+1) = b1(t) + learning_rate * (y(t) - yhat(t)) * yhat(t) * (1 - yhat(t)) * x1(t)</code>

El coeficiente especial al principio de la lista, también llamado intercepción, se actualiza de forma similar, excepto sin una entrada, ya que no está asociado a un valor de entrada específico:

<code>b0(t+1) = b0(t) + learning_rate * (y(t) - yhat(t)) * yhat(t) * (1 - yhat(t))</code>

Ahora podemos juntar todo esto. A continuación se muestra una función llamada <b>coefficients_sgd()</b> que calcula los valores de los coeficientes para un conjunto de datos de entrenamiento utilizando el descenso de gradiente estocástico.

In [ ]:
# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = row[-1] - yhat
            sum_error += error**2
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return coef

Puedes ver, que además, llevamos la cuenta de la suma del error al cuadrado (un valor positivo) de cada época para que podamos imprimir un bonito mensaje en cada bucle exterior.

Podemos probar esta función en el mismo pequeño conjunto de datos creado desde arriba.

In [ ]:
# Calculate coefficients
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

Utilizamos una mayor tasa de aprendizaje de 0,3 y entrenamos el modelo para 100 épocas, o 100 exposiciones de los coeficientes a todo el conjunto de datos de entrenamiento.

Al ejecutar el ejemplo se imprime un mensaje en cada época con la suma del error al cuadrado para esa época y el conjunto final de coeficientes.

In [ ]:
l_rate = 3
n_epoch = 100
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print(coef)

Puedes ver cómo el error sigue disminuyendo incluso en la época final. Probablemente podríamos entrenar durante mucho más tiempo (más épocas) o aumentar la cantidad que actualizamos los coeficientes en cada época (ritmo de aprendizaje más alto).

Experimente y vea lo que se le ocurre.

Ahora, apliquemos este algoritmo en un conjunto de datos real.

## 3. Predicción de la diabetes

En esta sección, entrenaremos un modelo de regresión logística utilizando el descenso de gradiente estocástico sobre el conjunto de datos de diabetes.

El ejemplo asume que una copia CSV del conjunto de datos está en el directorio de trabajo actual con el nombre de archivo nb4-diabetes.csv.

El conjunto de datos se carga primero, los valores de las cadenas se convierten a numéricos y cada columna se normaliza a valores en el rango de 0 a 1. Esto se logra con las funciones <code>helper load_csv()</code> y <code>str_column_to_float()</code> para cargar y preparar el dataset y <code>dataset_minmax()</code> y <code>normalize_dataset()</code> para normalizarlo.

Usaremos la validación cruzada de k-fold para estimar el rendimiento del modelo aprendido en datos no vistos. Esto significa que construiremos y evaluaremos los modelos k y estimaremos el rendimiento como el rendimiento medio del modelo. La exactitud de la clasificación se utilizará para evaluar cada modelo. Estos comportamientos se proporcionan en las funciones de ayuda <code>cross_validation_split()</code>, <code>accuracy_metric()</code> y <code>evaluate_algorithm()</code>.

Usaremos las funciones <code>predict()</code>, <code>coefficients_sgd()</code> creadas anteriormente y una nueva función <code>logistic_regression()</code> para entrenar el modelo.

A continuación se muestra el ejemplo completo.

In [ ]:
# Logistic Regression on Diabetes Dataset
from random import seed
from random import randrange
from csv import reader
from math import exp
 
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
# Make a prediction with coefficients
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-yhat))
n_folds = 5
l_rate = 0.1
n_epoch = 100
arreglo_epocas = [j for j in range(n_folds * n_epoch)]
arreglo_errores = []

# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        print("EPOCH:"+str(epoch))
        
        for row in train:
            
            yhat = predict(row, coef)
            #ESTE ES EL ERROR QUE HAY QUE GRAFICAR, en este preciso lugar: SEGRAFICA VS LA CANTIDAD DE FILAS EN ESTE LOOP
            error = row[-1] - yhat
            #print("ERROR:"+str(error))
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
            #print("coef_0:"+str(coef[0]))
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
        arreglo_errores.append(abs(error))
        print(abs(error))
        
    return coef
 
# Linear Regression Algorithm With Stochastic Gradient Descent
def logistic_regression(train, test, l_rate, n_epoch):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epoch)
    for row in test:
        yhat = predict(row, coef)
        yhat = round(yhat)
        predictions.append(yhat)
    return(predictions)
 
# Test the logistic regression algorithm on the diabetes dataset
seed(1)
# load and prepare data
filename = 'nb5-diabetes.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm

scores = evaluate_algorithm(dataset, logistic_regression, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Se utilizó un valor k de 5 para la validación cruzada, dando a cada muestra 768/5 = 153,6 o algo más de 150 registros para ser evaluados en cada iteración. Se eligió una tasa de aprendizaje de 0,1 y 100 épocas de entrenamiento con un poco de experimentación.

Puedes probar tus propias configuraciones y ver si puedes superar mi puntuación.

Al ejecutar este ejemplo, se imprimen las puntuaciones de cada uno de los 5 pliegues de validación cruzada y, a continuación, se imprime la precisión de clasificación media.

<b>Pregunta</b>:

Con Pandas y Matplotlib, dibujar una curva que representa la tasa de error de la regresión logística según el epoch.

In [ ]:
import matplotlib.pyplot as plt


df_errores = pd.DataFrame(arreglo_errores)
df_epocas = pd.DataFrame(arreglo_epocas)

plt.scatter(df_epocas, df_errores)

plt.show()